# 기사 제목 스크래핑 기초

BeautifilSoup검색 시 (단일/다중 element 검색)

soup.find(id=’id명’)\
soup.find(class=’class명’)\
soup.find(‘태그 이름’, class_=’class명’)\
soup.find_all(‘태그 이름’)

Selenium검색 시

find_element_by_id(‘id명’)\
find_element_by_class_name(‘class명’)\
find_element_by_xpath(‘xpath’)\
find_element_by_tag_name(‘tag명’)

import requests
from bs4 import BeautifulSoup as bs

url = "https://search.naver.com/search.naver?&where=news&query=유태오"

r = requests.get(url)
soup = BeautifulSoup(r.text,"html.parser")
news_titles = soup.select('ul.type01 > li')

for news in news_titles:
    title = news.select_one("a._sp_each_title").text
    date = news.select_one("dd.txt_inline").text
    
    print(title, date)

# 여러 페이지 가져오기

import urllib.request
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd

plusUrl = urllib.parse.quote_plus(input('검색어를 입력하세요:'))

pageNum = 1
count = 1

i = input('몇 페이지까지 크롤링 원해? :')

lastPage = int(i) * 10 - 9
while pageNum < lastPage + 1:
    url = f"https://search.naver.com/search.naver?&where=news&query={plusUrl}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=34&start=11&refresh_start={pageNum}"
    
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    title = soup.find_all(class_='_sp_each_title')
    
    print(f'------{count}페이지 결과입니다.-----')
    
    for i in title:
        print(i.attrs['title'])
        print(i.attrs['href'])
        print()
    
    pageNum += 10
    count += 1    

def get_news(query):
    
    url_query = quote(query)
    url = "https://search.naver.com/search.naver?&where=news&query={}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=34&start=11&refresh_start={}".format('plusurl','pageNum') + url_query
    
    search_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(Search_url, 'html.parser')
    links = soup.find_all('dd', {'class':'txt_inline'})
    
    for link in links:
        news_date = link.find('span',{'class':'bar'}).get_text()
        news_title = link.find('a').get('class')

# -------지금부터 시작합니다 -----

## 연예기사는 무조건 selenium 사용해야 하ㅏㅏ... 

In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [2]:
from selenium import webdriver

In [3]:
d = webdriver.Chrome("c:/Chrome/chromedriver.exe")

In [4]:
d.get("https://www.naver.com") # 네이버 입장

In [5]:
elem = d.find_element_by_id("query") # 검색어를 어디에서 입력하지

In [6]:
elem.send_keys("유태오") # 검색어를 입력한다 꺄앙
elem = d.find_element_by_id("search_btn")
elem.click() # 검색어 클릭

In [7]:
d.find_element_by_css_selector("#lnb > div > div.lnb_menu > ul > li.lnb4").click() #뉴스 클릭!

In [8]:
d.find_element_by_css_selector('#_search_option_btn').click() # 검색옵션 

In [9]:
d.find_element_by_css_selector('#snb > div > ul > li:nth-child(4) > a').click() # 기간클릭

In [10]:
d.find_element_by_css_selector('#_nx_option_date > div.group_choice > ul:nth-child(1) > li:nth-child(4) > a').click() # 1개월!!

In [11]:
# 신문사와 날짜 추출

html = d.page_source
bs_naver = bs(html, 'html.parser')
news_titles = bs_naver.select('ul.type01 > li')
    
import pandas as pd
total = pd.DataFrame()
    
for news in news_titles:
    title = news.select_one("a._sp_each_title").text
    date = news.select_one("dd.txt_inline").text
    hhh = (title +'  '+ date).split('  ')[:3]

    total = total.append(pd.DataFrame({ "제목":[hhh[0]] , "신문사":[hhh[1]] ,"발행일" : [hhh[2]] }))
total

Nextpage = d.find_element_by_css_selector("#main_pack > div > div.paging > a:nth-child(2)").click()

for page in range(1,11):
    try:
        html = d.page_source
        bs_naver = bs(html, 'html.parser')
        news_titles = bs_naver.select('ul.type01 > li')


        for news in news_titles:
            title = news.select_one("a._sp_each_title").text
            date = news.select_one("dd.txt_inline").text
            hhh = (title +'  '+ date).split('  ')[:3]

            total = total.append(pd.DataFrame({ "제목":[hhh[0]] , "신문사":[hhh[1]] ,"발행일" : [hhh[2]] }))

        # 페이지 이동하기
        d.find_element_by_xpath('//*[@id="main_pack"]/div/div[2]/a[%s]' %page).click()
        
    except:
        break

In [13]:
total = total.reset_index(drop=True)
total

,제목,신문사,발행일
0,"유태오, 넷플릭스 '보건교사 안은영' 영어교사 역 발탁",뉴시스,2020.08.20.
1,부산구치소 복지시설 위문품,부산일보,14시간 전
2,부산구치소장에 우희경 교정본부 교정기획과장,로이슈,20시간 전
3,"부산구치소, 직원성금으로 추석맞이 사회복지시설 4곳 위문",로이슈,1일 전
4,"부산제일라이온스클럽, 코로나19 예방 면마스크 2300장 부산구치소에 기증",로이슈,2일 전
5,"부산구치소, 엄궁중·동주중에 교정장학금 전달",파이낸셜뉴스,2020.09.08.
6,"권영찬, 생애 첫 오디션으로 넷플릭스 ‘보건교사 안은영’으로 주조연 데뷔!",MBC연예,3일 전
7,"권영찬, 넷플릭스 '보건교사 안은영' 캐스팅",톱데일리,3일 전
8,"[포토] 부산구치소 직원들, 사상구 관내 중학생에게 장학금 전달",시장경제신문,2020.09.09.
9,"""아, XX""··· '보건교사 안은영' 예고편에 욕설 그대로 노출",서울경제,2020.08.25.


In [ ]:
total.to_excel('나이와결혼여부.xlsx', index=False)